# Synthesizer
### Arnaud Gaudard, Julien Sahli, Corentin Junod

This notebook presents our projet functionalities.
The synthesizer comes also with a GUI showing the output frequencies in real time, it is not displayed in this notebook. To use the program with all its functionalities, please run it as mentioned in the README. 

All the requirements for the project to run are listed in `requirements.txt`. To install them run `pip install -r requirements.txt`

When a cell is run, the synth activates and start playing. To stop it, press "Enter".


In [1]:
from multiprocessing import Queue, Process
import pygame.midi
import mido
import argparse
import numpy as np
import sounddevice as sd
from Synthesizer import Synthesizer
from FilePlayer import FilePlayer
from GUI import GUI
from utils import show_peripherals
import current_script
import parameters

sd.default.device = parameters.OUTPUT_DEVICE
sd.default.samplerate = parameters.SAMPLING_FREQUENCY
mido.set_backend('mido.backends.pygame')
pygame.midi.init()
midi_input = pygame.midi.Input(parameters.INPUT_MIDI_DEVICE)

pygame 2.1.2 (SDL 2.0.16, Python 3.10.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
"""
Basic function that runs the synth.
It is similar to cli_main.py
"""
def start_synth(MidiToFreq=current_script.MidiToFreq, FreqToAudio=current_script.FreqToAudio):
    global midi_input
    queue = Queue()
        
    current_script.MidiToFreq = MidiToFreq
    current_script.FreqToAudio = FreqToAudio
    current_script.miditofreq = current_script.MidiToFreq()
    current_script.freqtoaudio = current_script.FreqToAudio()

    # This file contains a medley of various Mario musics, absolutely perfect for tests
    # If you're not part of the Nintendo fanbase, you may use midi_example2.mid, which is more baroque
    file_player = FilePlayer('ressources/midi_example.mid')
    file_player_proc = Process(target=file_player.run)

    synth = Synthesizer(queue, midi_input)
    
    # In this notebook, we do not run the GUI
    # gui = GUI(queue)
    # proc = Process(target=gui.run).start()

    file_player_proc.start()
    synth.run()

show_peripherals()

PyGame MIDI Inputs : 
   1 : ALSA - Midi Through Port-0             Opened: yes
   3 : ALSA - out                             Opened: no


Mido MIDI Outputs : 
['Midi Through Port-0', 'in']


Audio Outputs : 
   0 HDA Intel PCH: ALC289 Analog (hw:0,0), ALSA (2 in, 4 out)
   1 HDA Intel PCH: HDMI 0 (hw:0,3), ALSA (0 in, 8 out)
   2 HDA Intel PCH: DELL U2419H (hw:0,7), ALSA (0 in, 2 out)
   3 HDA Intel PCH: HDMI 2 (hw:0,8), ALSA (0 in, 8 out)
   4 HDA Intel PCH: HDMI 3 (hw:0,9), ALSA (0 in, 8 out)
   5 HDA Intel PCH: HDMI 4 (hw:0,10), ALSA (0 in, 8 out)
   6 G433 Gaming Headset: USB Audio (hw:1,0), ALSA (2 in, 0 out)
   7 sysdefault, ALSA (128 in, 0 out)
   8 spdif, ALSA (2 in, 0 out)
   9 lavrate, ALSA (128 in, 0 out)
  10 samplerate, ALSA (128 in, 0 out)
  11 speexrate, ALSA (128 in, 0 out)
  12 pipewire, ALSA (64 in, 64 out)
* 13 pulse, ALSA (32 in, 32 out)
  14 speex, ALSA (1 in, 0 out)
  15 upmix, ALSA (8 in, 0 out)
  16 vdownmix, ALSA (6 in, 0 out)
  17 default, ALSA (128 in, 0 out

<hr style="border-color:black">

## Basic principles

Our synthesizer takes MIDI input in real time or from a file, and convert them into sound. 

The steps are shown in the following diagram. 

<img src="ressources/Process.png"/>

Each step in further detailed in this document.

## Basic example

This is a first basic example generating a simple sine oscillator for each note

In [ ]:
import Modules.Oscillators
"""
This class tells the synth how to convert tuples of frequencies and amplitudes into sound.

It must implement a constructor, where all the oscillators and filters must be declared, it 
is called once when the synth starts.

The method "process" is called for each audio frame.

"""
class FreqToAudio:
    def __init__(self):
        # Oscillators
        self.sine = Modules.Oscillators.Sine()

    """
    Converts a list of frequencies and amplitudes into sound.
    Parameters :
        indexes : An array of indexes of size parameter.SAMPLES_PER_FRAME. 
                  Indicates the indexes of the samples produced by the function, starting from
                  0 for the first sample produced and incrementing over time. This is used by
                  filters operating over time.
        freqs_amps : An array of tuples (frequency, amplitude) that contains en entry per 
                     frequency in herz that must be generated, associated with it amplitude (loudness)
                     from 0 to 1
                     
    Return value :
        A tuple of two arrays (of length parameter.SAMPLES_PER_FRAME) containing the generated 
        samples. The first array is the left ear audio, the second is the right ear audio.
    """
    def process(self, indexes, freqs_amps):
        output = np.zeros(parameters.SAMPLES_PER_FRAME)
        
        # There is no filtering of frequencies and amplitudes in this example
        
        # Oscillators
        for freq, amp in freqs_amps:
            # Security cutting frequencies over the Nyquist frequency
            if freq > parameters.NYQUIST_FREQUENCY: continue
            output += self.sine.set(freq, amp=amp).get(indexes, output)
            
        # There is no filtering of audio signal in this example
        
        # This example is mono
        return output, output 
    
start_synth(FreqToAudio=FreqToAudio)

<hr style="border-color:black">

## Piano-like sound

This is an example of a very basic piano sound, to illustrate more functonalities.

In [4]:
import Modules.Oscillators
import Modules.filters.Reverb
import Modules.ADSR
import Modules.Linear
import Modules.Exponential

class FreqToAudio:
    def __init__(self):
        # Filters
        self.reverb = Modules.filters.Reverb.Reverb(delay=0.005, dampening=0.3)

        # Oscillators
        self.sine = Modules.Oscillators.Sine()

        # ADSR
        attack_time = 0.005
        attack_stop_level = 0.9
        attack_func = Modules.Linear.Linear(start=0, stop=attack_stop_level, duration=attack_time)
        decay_time = 3
        decay_func=Modules.Exponential.Exponential(start=attack_stop_level, stop=0, duration=decay_time)
        release_time = 0.1
        release_func = Modules.Linear.Linear(attack_stop_level, 0, release_time)
        self.adsr = Modules.ADSR.ADSR(
            attack_time=attack_time, attack_func=attack_func,
            decay_time=decay_time,   decay_func=decay_func,
            sustain_func=0,
            release_time=release_time, release_func=release_func,
        )

    def process(self, indexes, freqs_amps) :
        output = np.zeros(parameters.SAMPLES_PER_FRAME)

        # Frequencies filtering - ADSR
        freqs_amps = self.adsr.get(indexes, freqs_amps)

        # Oscillators
        # overtones taken from a C4 on a piano
        # the tuples are [overtone_number, amplitude]
        overtones = [
            [1, 1],
            [2, 0.9],
            [3, 0.15],
            [4, 0.39],
            [5, 0.39],
            [6, 0.1],
            [7, 0.2],
            [9, 0.15]
        ]
        for freq, amp in freqs_amps:
            if freq > parameters.NYQUIST_FREQUENCY: continue
            for over_mult, over_amp in overtones:
                output += self.sine.set(freq*over_mult, amp=amp*over_amp).get(indexes, output)

        # Audio filtering
        output = self.reverb.get(indexes, output)

        return output, output
    
start_synth(FreqToAudio=FreqToAudio)

Unable to open Midi OutputDevice=0: b"PortMidi: `Host error'"


ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

...

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun occurred
ALSA lib pcm.c:8568:(snd_pcm_recover) underrun o

KeyboardInterrupt: 